In [164]:
import os
import json
import torch
import warnings
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
warnings.simplefilter('ignore')

In [165]:
### load title name entity tags and short hand token
IMPROT_FILE = 'validData_Reduced.csv'
ETET_GRAPHS = 'TemporalGraphData\ETETGraphs'
STANDARD_TITLE_FILES = ['tech_jobs.csv', 'non_tech_jobs.csv']
IMPORT_PATH = ''

In [166]:
### Deal with Titles
with open(os.path.join(IMPORT_PATH, "title_id_dict.json"), "r") as openfile:
    org_title_2_new_id = json.load(openfile)
with open(os.path.join(IMPORT_PATH, "employee_id_dict.json"), "r") as openfile:
    org_employee_id_2_new_id = json.load(openfile)
# load standard titles
df_titles = pd.concat([pd.read_csv(os.path.join(IMPORT_PATH, STANDARD_TITLE_FILES[0])),  pd.read_csv(os.path.join(IMPORT_PATH, STANDARD_TITLE_FILES[1]))]).drop(columns=['Edu'])
df_titles.GROUP = df_titles.GROUP.str.lower(); df_titles.TITLE = df_titles.TITLE.str.lower()
# save mapping
org_title_gp_2_new_id = {each : 'G' + str(idx) for idx, each in enumerate(df_titles.GROUP.unique())}
df_titles['GroupID'] = df_titles.GROUP.apply(lambda x: org_title_gp_2_new_id[x])
df_titles['TitleID'] = df_titles.TITLE.apply(lambda x: org_title_2_new_id[x] if x in org_title_2_new_id else np.nan)
df_titles.dropna(subset=['TitleID'], inplace=True)
with open(os.path.join(IMPORT_PATH, "title_gp_id_dict.json"), "w") as outFile:
    json.dump(org_title_gp_2_new_id, outFile)
df_title_hierachy = df_titles[['GroupID', 'TitleID']]
### import title embedding matrix
z_raw = np.load(os.path.join(IMPORT_PATH, 'title_emb_matrix.npy'))

In [170]:
### import golden source (no duplicates)
df = pd.read_csv(os.path.join(IMPORT_PATH, IMPROT_FILE))
df['Start_Year'] = df['Start_Date'].apply(lambda x: int(x.split('-')[0]))
df['End_Year'] = df['End_Date'].apply(lambda x: int(x.split('-')[0]))
df.drop_duplicates(subset=['ID', 'End_Year'], keep='last', inplace=True)
df.drop_duplicates(subset=['ID', 'Start_Year'], keep='last', inplace=True)
df['EmployeeID'] = df.ID.apply(lambda x: org_employee_id_2_new_id[x])
df['TitleID'] = df.Title.apply(lambda x: org_title_2_new_id[x])
df['GroupID'] = df.Title_Group.apply(lambda x: org_title_gp_2_new_id[x])
df['Job_Index'] = df.groupby('EmployeeID').cumcount()
df.head(3)

,ID,Firm,Title,Start_Date,End_Date,Title_Group,EDU,Duration,DiversityScore,YrsOfExp,Start_Year,End_Year,EmployeeID,TitleID,GroupID,Job_Index
0,"ACwAAA--S20BZncfI96Y51rtML5hkDoodwbFi-c,NAME_S...",Amazon,software engineer,2019-12-01,2021-10-01,software developer,"computer science, computer engineering, electr...",2,0.000000,1.0,2019,2021,E1,T1,G0,0
1,"ACwAAA--zVABrailb54YsBsNL_ulhYNRVeX599Y,NAME_S...",Amazon,software engineer,2018-03-01,2019-03-01,software developer,"computer science, computer engineering, electr...",1,0.000000,0.0,2018,2019,E2,T1,G0,0
2,"ACwAAA--zVABrailb54YsBsNL_ulhYNRVeX599Y,NAME_S...",JT4,technical analyst,2020-01-01,2020-07-01,it specialist,information technology,1,0.693147,1.0,2020,2020,E2,T2,G8,1


In [182]:
### number of experiences distribution
df_count = df.groupby('ID')[['Firm']].count().reset_index().rename(columns={'Firm' : 'Count'})
# display(df_count.Count.describe())
### sample data ( lb < num exps < ub)
lb, ub = 3, 10
v = df_count[(df_count.Count >= lb)&(df_count.Count <= ub)]
print(f'There are {len(v)} employees that have experience between {lb}~{ub}.')
df_sampled = df[df.ID.isin(set(v.ID))].reset_index(drop=True)
unique_gps = df_sampled.Title_Group.unique()
unique_titles = df_sampled.Title.unique()
df_sampled = df_sampled[
    ['EmployeeID', 'TitleID', 'GroupID', 'Start_Year', 'End_Year', 'Duration', 'DiversityScore', 'Job_Index']]

There are 19377 employees that have experience between 3~10.


In [177]:
### ETET network
tmp_col = []
path = os.path.join(IMPORT_PATH, ETET_GRAPHS)
for yr in range(2008, 2023):
    df_tmp = pd.read_csv(os.path.join(path, 'graph_etet_{yr}.csv'.format(yr=yr)))
    df_tmp['Year'] = yr
    tmp_col.append(df_tmp)
df_etet = pd.concat(tmp_col, axis=0)[['Year', 'Focal', 'Reference', 'Start_Year', 'Job_Index', 'TargetTitle']]
# path = os.path.join(IMPORT_PATH, ETET_GRAPHS)
# df_etet.to_csv(os.path.join(path, 'AggregatedETET.csv'), index=False)

Create Data Point:
- X:
    - EmployeeID    
    - Num of Exp
    - End_Year
    - Job_Index
    - Duration
    - Diversity Score
    - Title Embedding    
Format:
    1 : [EmployeeID, NumExp, End_Year, Dur, Div, titleEmb]
    2 : [EmployeeID, NumExp, End_Year, Dur, Div, titleEmb]
    ... 
    if not as many as upper bound, pad zeros
- Y:
    - TitleID


In [184]:
row

EmployeeID         E10
TitleID            T11
GroupID            G30
Start_Year        2008
End_Year          2008
Duration             1
DiversityScore     0.0
Job_Index            0
Name: 7, dtype: object

In [185]:
### we shall pad a full seq starting from 2008 -> 2022
count = 0
x, y = [], []
gps = df_sampled.groupby('EmployeeID')
for id, profile in gps:
    # below two attributes are shared across one profile
    employee_id = int(id.split('E')[1]) # employee id
    num_exp = len(profile) # career length
    x_col, y_col = [], []
    job_count = 0
    for _, row in profile.iterrows():
        dur = row.Duration # duration
        end_yr = row.End_Year # end year
        job_idx = row.Job_Index
        title_id = int(row.TitleID.split('T')[1])-1 # title id
        t_emb = z_raw[title_id] # title embedding
        div = row.DiversityScore # diversity score
        x_col.append([employee_id, num_exp, end_yr, job_idx, dur, div] + t_emb.tolist())
        y_col.append([title_id])
    # x for one profile
    x_stack = np.stack(x_col)
    x.append(np.concatenate([x_stack, np.zeros(((ub - num_exp), len(x_col[0])))], axis=0))
    # y for one profile
    y_stack = np.stack(y_col)
    y.append(np.concatenate([y_stack, np.zeros(((ub - num_exp), len(y_col[0])))], axis=0))
    count += 1
    if count % 100 == 0:
        print(count)
# turn np to tensor
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [186]:
### split
num_samples = len(x_tensor)
test_samples = round(0.1 * num_samples)
test_idx = np.random.choice(list(range(num_samples)), test_samples)
x_test_tensor = x_tensor[test_idx]
y_test_tensor = y_tensor[test_idx]
train_idx = list(set(range(num_samples)) - set(test_idx))
x_train_tensor = x_tensor[train_idx]
y_train_tensor = y_tensor[train_idx]

In [187]:
### save data set
export_path = os.path.join(IMPORT_PATH, 'TemporalGraphData\TrainTestData')
torch.save(x_train_tensor, os.path.join(export_path, 'TRAIN_X.pt'))
torch.save(y_train_tensor, os.path.join(export_path, 'TRAIN_Y.pt'))
torch.save(x_test_tensor, os.path.join(export_path, 'TEST_X.pt'))
torch.save(y_test_tensor, os.path.join(export_path, 'TEST_Y.pt'))
# debug data set
torch.save(x_train_tensor[:100], os.path.join(export_path, 'DEBUG_X.pt'))
torch.save(y_train_tensor[:100], os.path.join(export_path, 'DEBUG_Y.pt'))
torch.save(x_test_tensor[:100], os.path.join(export_path, 'DEBUG_TEST_X.pt'))
torch.save(y_test_tensor[:100], os.path.join(export_path, 'DEBUG_TEST_Y.pt'))

In [189]:
### data loader
class CustomDataSet(Dataset):

    def __init__(self, root_path, x_file, y_file, title_emb_dim=384, use_cuda=False):
        self.dataset = torch.load(os.path.join(root_path, x_file),)
        self.labels = torch.load(os.path.join(root_path, y_file))
        self.title_emb_dim = title_emb_dim
        if use_cuda:
            self.dataset = self.dataset.cuda()
            self.labels = self.labels.cuda()
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        ### Reminder
        # Create Data Point:
        # - X:
        #     - EmployeeID    
        #     - Num of Exp
        #     - End_Year
        #     - Job_Index
        #     - Duration
        #     - Diversity Score
        #     - Title Embedding

        # Format:
        #     1 : [EmployeeID, NumExp, End_Year, Dur, Div, titleEmb]
        #     2 : [EmployeeID, NumExp, End_Year, Dur, Div, titleEmb]
        #     ... 
        #     if not as many as upper bound, pad zeros
        # - Y:
        #     - TitleID
        x, y = self.dataset[idx], self.labels[idx]
        emp_id = x[:, 0]
        num_exp = x[:, 1:2]
        end_yr = x[:, 2:3]
        job_idx = x[:, 3:4]
        dur = x[:, 4:5]
        div = x[:, 5:6]
        t_emb = x[:, 6:]
        return emp_id[0], num_exp[0], end_yr, job_idx, dur, div, t_emb, y[:, 0]

In [190]:
### test
import_path = os.path.join(IMPORT_PATH, 'TemporalGraphData\TrainTestData')
title_emb_dim = z_raw.shape[1]
train_dataset = CustomDataSet(import_path, 'TRAIN_X.pt', 'TRAIN_Y.pt', title_emb_dim)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

In [191]:
for i, (emp_id, num_exp, end_yr, job_index, dur, div, t_emb, label) in enumerate(train_dataloader):
    break
    if i == 100:
        break